# dotQ challenge Nº1: $|\textit{FizzBuzz}\rangle$
### By: Enrique Anguiano-Vara 

- Every programmer knows the FizzBuzz programming interview question: 
> "Given a list of items from 1 to 100, print on the screen 'Fizz' if the iteration number is a multiple of 3 and 'Buzz' if it is 5. Of course, if a number is a multiple of 3 and 5, it will print 'FizzBuzz'" 

- Coding it in a classical way is easy. Here is my example in C++ (if you don't know this programing languaje, [here](https://web.stanford.edu/class/archive/cs/cs106b/cs106b.1178/lectures/28-DifferentLanguages/code/handout.pdf) is the solution it in Python, JavaScript, Haskell and COBOL):
```
#include <iostream>

using namespace std;

int main ()
{
    for (int i = 1; i <= 100; i++)
    {
        cout << i << ": ";
        if (!(i%3)) cout << "Fizz";
        if (!(i%5)) cout << "Buzz";
        cout << endl;
    }
  return 0;
}

```
- But can you do it using a quantum computing tool?


## Let's make it quantum!
Now we will modify this exercise a bit. First, instead of a list we will have single number, $x$, that will be a function parameter. The main idea, using the fizzbuzz operator, $U_{FB}$:

$$U_{FB}(x)|0\rangle^{\otimes 2} \Rightarrow |\textit{is_Fizz}\rangle|\textit{is_Buzz}\rangle$$

or you can use x as a input state, $|x\rangle$:

$$U_{FB}|x\rangle^{\otimes n}|0\rangle^{\otimes 2} \Rightarrow |x\rangle^{\otimes n} |\textit{is_Fizz}\rangle|\textit{is_Buzz}\rangle$$

The decision is yours :)

### What is the challenge?
- Make a quantum circuit equivalent to $U_{FB}$
  - In this challenge we'll use **Qiskit**
  - Complete this exercise correctly for **any x**. 
  - You'll be competing with others to win. We'll have **two types of winners**: the **fastest** and the one with the **best idea**. Try to win both!

In [89]:
# Import libraries
from qiskit import QuantumCircuit
import numpy as np

In [90]:
def fizzbuzz(x : int) -> QuantumCircuit:
    '''
    INPUTS: x, the input number
    OUTPUTS: the fizzbuzz quantum ciruit
    '''
    pass
    

## My solution


In [91]:
import numpy as np
import math
from qiskit.quantum_info import Operator
from qiskit.circuit.library import MCMT,XGate
from qiskit import QuantumCircuit

In [92]:
def x_mod_y(x, y):
    N = 2 ** int(math.ceil(math.log(y, 2)))
    u = np.zeros(shape = (N,N))
    x_mod_y = x % y
    
    permutation = 0
    for i in range(x_mod_y, (y + x_mod_y)):
        u[i % y][permutation] = 1
        permutation += 1
    
    for r in range(y,N):
        u[r][r] = 1
    
    U = Operator(u)
    return U

In [99]:
def fizz(qc,x):
    qc.append(x_mod_y(x,3),[5,6])
    qc.x([5,6])
    qc.append(MCMT(XGate(),2,1),[5,6,1])

def buzz(qc,x):
    qc.append(x_mod_y(x,5),[2,3,4])
    qc.x([2,3,4])
    qc.append(MCMT(XGate(),3,1),[2,3,4,0])

In [100]:
def fizzbuzz(x: int)-> QuantumCircuit:
    qc = QuantumCircuit(2+3+2,2) # (isFizz,is_Buzz)+ ancilla for Fizz + ancilla for Buzz
    fizz(qc,x)
    buzz(qc,x)
    qc.measure([0,1],[0,1])
    return qc

### Now let's see if this works

In [101]:
from qiskit_ibm_runtime import Estimator
from qiskit.quantum_info import SparsePauliOp
from qiskit import Aer, execute

tests = [4,33,50,15] #Correct ouputs: 00,10,01,11
execute([fizzbuzz(x) for x in tests], Aer.get_backend("qasm_simulator")).result().get_counts()

[{'00': 1024}, {'10': 1024}, {'01': 1024}, {'11': 1024}]

### Circuit visualization

In [102]:
print(fizzbuzz(0))

                               ┌───────┐┌─┐   
q_0: ──────────────────────────┤3      ├┤M├───
                      ┌───────┐│       │└╥┘┌─┐
q_1: ─────────────────┤2      ├┤       ├─╫─┤M├
     ┌──────────┐┌───┐│       ││       │ ║ └╥┘
q_2: ┤0         ├┤ X ├┤       ├┤0 mcmt ├─╫──╫─
     │          │├───┤│       ││       │ ║  ║ 
q_3: ┤1 Unitary ├┤ X ├┤       ├┤1      ├─╫──╫─
     │          │├───┤│  mcmt ││       │ ║  ║ 
q_4: ┤2         ├┤ X ├┤       ├┤2      ├─╫──╫─
     ├──────────┤├───┤│       │└───────┘ ║  ║ 
q_5: ┤0         ├┤ X ├┤0      ├──────────╫──╫─
     │  Unitary │├───┤│       │          ║  ║ 
q_6: ┤1         ├┤ X ├┤1      ├──────────╫──╫─
     └──────────┘└───┘└───────┘          ║  ║ 
c: 2/════════════════════════════════════╩══╩═
                                         0  1 
